# Beispiel Dimensionsreduktion und Datenvisualisierung

Um die Dimensionsreduktion ausprobieren zu können, schaust du dir am besten ein Beispiel an. Da du dir viele Dimensionen nur schwer vorstellen kannst, wirst du alles auf zwei Dimensionen reduzieren und intensiv mit Visualisierungen arbeiten.

## Das `digits`-Datenset

Hierzu nutzt du das `digits`-Datenset, das bereits in `scikit-learn` integriert ist. Dabei handelt es sich um handgeschriebene Ziffern. Die Bilder sind dabei auf 8\*8 Pixel reduziert. Jeder Pixel kann Grauwerte von `0` bis `15` annehmen. Es sind insgesamt 1.797 Bilder.

Das Datenset kannst du ganz einfach laden:

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()

Du könntest dir nun die Grauwerte anzeigen lassen, aber dabei kannst du nicht viel erkennen. Es ist einfacher, wenn du dir ein paar der Bilder ausgeben lässt:

In [ ]:
import matplotlib.pyplot as plt
for i in [0, 200, 400, 600]:
    print(digits.target[i])
    plt.gray()
    plt.matshow(digits.images[i])
    plt.show()

Mit etwas Fantasie und aus größerer Entfernung kannst du die Ziffern erkennen.

Nun konvertierst du die Daten in einen `DataFrame`:

In [ ]:
import pandas as pd
df = pd.DataFrame(digits["data"], columns=digits["feature_names"])
df["class"] = [digits["target_names"][target] for target in digits["target"]]
df

Du erkennst die Anzahl der Samples und die Features (`target` zählt nicht als Feature.

Ein bisschen Statistik kann nicht schaden:

In [ ]:
df.describe()

Die obere Zeile scheint nicht stark beschrieben zu sein, sonst kannst du nicht sehr viel erkennen.

## PCA

Als erstes probierst du PCA, also die Hauptkomponentenanalyse aus. Ein Charakteristikum der PCA sind die Eigenwerte. Anhand dieser kannst du entscheiden, auf wie viele Dimensionen du mehr oder weniger *gefahrlos* reduzieren kannst.

Um alle Eigenwerte zu bestimmen, kannst du die Dimensionen gleich lassen:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=digits["data"].shape[1]).fit(digits["data"])

Die Eigenwerte kannst du nun plotten. Der sog. Scree-Plot zeigt die, wie viele Dimensionen du benötigst:

In [ ]:
plt.plot(pca.singular_values_)

Das wären bei PCA schon sehr viele Dimensioenen, also sicher 40-50. Das hilft dir hier nicht viel weiter, also probierst du es mit zwei Dimensionen:

In [ ]:
pca2 = PCA(n_components=2)

In `scikit-learn` findet die Transformation immer mit der Methode `fit_transform` statt. Das Ergebnis überträgst du gleich in einen `DataFrame` und packst das Target noch mit dazu:

In [ ]:
pdf = pd.DataFrame(pca2.fit_transform(digits["data"]), columns=["x", "y"])
pdf["target"] = digits["target"]

Den so entstandenen `DataFrame` kannst du mit einem *Scatterplot* visualisieren. Leider verschiebt `matplotlib` die Skala etwas. Das lässt sich nur mit erheblichem Aufwand korrigieren und gehört eher in einen Visualisierungskurs. Also sei bitte vorsichtig, wenn du die Legende abliest.

In [ ]:
pdf.plot.scatter(x='x', y='y', c="target", figsize=(10,10), cmap="tab10")

Manche Zahlen konnte PCA schon gut voneinander trennen, bei anderen hat das gar nicht geklappt und es gibt große Überlagerungen. Das Ergebnis ist nicht wirklich brauchbar.

## t-SNE

Nun kennst du die Aufrufe von `scikit-learn` schon und kannst nach dem identischen Schema eine Dimensionsreduktion mit t-SNE durchführen:

In [ ]:
from sklearn.manifold import TSNE
tsne2 = TSNE(n_components=2, random_state=42)
tdf = pd.DataFrame(tsne2.fit_transform(digits["data"]), columns=["x", "y"])
tdf["target"] = digits["target"]

Das dauert eine Weile länger, ist das Ergebnis besser?

In [ ]:
tdf.plot.scatter(x='x', y='y', c=tdf["target"], figsize=(10,10), cmap="tab10")

Das ist ein sehr großer Unterschied! Die einzelnen Ziifern sind sehr deutlich voneinander getrennt, nur die `1` tanzt etwas aus der Reihe. `3` und `9` überlappen sich leicht - das kann einer schlampigen Schrift zugeschrieben werden.

## UMAP

PCA wird doch noch relativ häufig verwendet, t-SNE dagegen eher selten. Es ist langsamer und skaliert nicht gut. Außerdem funktioniert es ähnlich wie UMAP, das deutlich schneller ist. Das probierst du jetzt aus. Leider ist UMAP noch nicht in `scikit-learn` integriert, du musst es manuell installieren:

In [ ]:
!pip install umap-learn[parametric_umap]

Zum Glück ist der Aufruf aber sehr ähnlich (es erbt einige Klassen von `scikit-learn`):

In [ ]:
import umap
umap2 = umap.UMAP(n_components=2, random_state=42)
udf = pd.DataFrame(umap2.fit_transform(digits["data"], ), columns=["x", "y"])
udf["target"] = digits["target"]

Wie sieht das Ergrebnis hier aus?

In [ ]:
udf.plot.scatter(x='x', y='y', c="target", figsize=(10,10), cmap="tab10")

Die Daten sind noch deutlich besser separiert. In diesem Fall ist UMAP unser "Sieger". Das ist häufig so. UMAP ist in den meisten Fällen das heute beste Verfahren, um Dimensionen zu reduzieren

## Dimensionreduktion als erster Schritt zum Machine Learning

Häufig wirst du mit hochdimensionalen Daten arbeiten müssen. Das ist aber für viele Anwendungen nicht geschickt. Daher solltest du versuchen, die Dimensionen zu reduzieren oder zumindest zu überprüfen, ob das gut funktioniert. PCA ist immer einen Versuch wert, aber in echten Projekten wirst du sicher am häufigsten mit UMAP arbeiten.

t-SNE ist hauptsächlich historisch interessant, weil es 2008 einen neuen Ansatz geboten hat. UMAP ist hier fast immer überlegen